In [13]:
from transformers import AutoTokenizer
from trl import AutoModelForCausalLMWithValueHead
import torch

# モデルとトークナイザのロード
model_name = 'rinna/japanese-gpt-1b'  # 日本語モデル
model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

# GPUを使用可能であれば設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 入力プロンプト
prompt = "AIを使った新しいアイデアについて教えてください。"

# トークン化（入力テキストをモデルに渡せる形式に変換）
tokenized_input = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128)
input_ids = tokenized_input.input_ids.to(device)  # GPUに移行

# テキスト生成のパラメータ
generation_kwargs = {
    "max_length": 1000,       # 最大生成トークン数
    "temperature": 0.7,      # 多様性の調整（値が大きいほどランダム性が増加）
    "top_k": 50,             # 上位50トークンから選択
    "top_p": 0.9,            # 確率の累積が0.9を超えるトークンから選択
    "do_sample": True,       # サンプリングを有効化
    "pad_token_id": tokenizer.eos_token_id  # パディングトークンID
}

# モデルで生成
output_ids = model.generate(input_ids, **generation_kwargs)

# トークンをデコードして人間が読める形式に変換
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 結果を表示
print("生成されたテキスト:")
print(output_text)

生成されたテキスト:
AIを使った新しいアイデアについて教えてください。 「AIによる新しいアイデアを思いついた」という声は、多く聞きます。ただ、AIは、人間の脳ほど、論理的思考が得意ではありません。AIは、与えられた課題に対して、人間ならどう考えるかという視点から、さまざまな仮説を立て、それを検証していきます。仮説を検証する中で、人間なら思いつかないようなアイデアが生まれるのです。


In [20]:
question = ''

tokenized_input = tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=128)
input_ids = tokenized_input.input_ids.to(device) 
output_ids = model.generate(input_ids, **generation_kwargs)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(output_text)

生成されたテキスト:
強化学習に関する新しい研究を考えてください 強化学習に関する新しい研究を考えてください。あなたは、強化学習のアルゴリズムの新しい実装を考案し、その実装が新しい問題をどのように解決し、そしてどのように新しいアルゴリズムが新しい問題を解くことができるかを説明してください。


In [15]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "GeneZC/MiniChat-1.5-3B"  # 使用したいモデル名を指定
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# # テキスト生成の例
# input_text = "強化学習とLLMに関する研究アイデアを作成してください"
# inputs = tokenizer(input_text, return_tensors="pt")
# outputs = model.generate(inputs["input_ids"])
# generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# print(generated_text)

In [21]:
import g4f
question = ''
response = g4f.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "user", "content": question}],
)
print(response)

以下に、いくつかの興味深い強化学習の応用アイデアを提案します：

1. 個人最適化学習システム
• 学習スタイル
• 健康管理
• キャリア開発

2. 都市交通最適化
• リアルタイム信号制御
• 自動運転車の協調
• エネルギー効率最適化

3. 創造的AI支援
• 音楽作曲
• デザイン生成
• ストーリー創作支援

4. 環境適応ロボット
• 災害救助
• 未知環境探索
• 農業自動化

5. パーソナライズド医療
• 治療計画最適化
• 薬剤投与調整
• リハビリテーション支援

6. サイバーセキュリティ
• 異常検知
• リアルタイム防御
• ネットワーク保護

これらのアイデアは、強化学習の可能性を示唆しています。


# 配列（data）の作成

In [23]:
def get_output_by_g4f(question: str) -> str:
    """
    y=1のときの出力を取得する関数
    """
    response = g4f.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": question}],
    )
    # `response` が辞書やオブジェクトになる可能性があるため、
    # 文字列として取り出せる形をとっています
    return str(response)

def get_output_by_local_model(question: str) -> str:
    """
    y=0のときの出力を取得する関数
    """
    # tokenizer, model, device, generation_kwargs はグローバルにある想定
    # 必要に応じて関数内で参照できるようにしてください
    tokenized_input = tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=128)
    input_ids = tokenized_input.input_ids.to(device)
    output_ids = model.generate(input_ids, **generation_kwargs)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

In [25]:
x = []
y = []   
num_iterations = 100
question = "LLMを使った新しいアプリのアイデアをいくつか出してください"
    
for i in range(num_iterations):
    print(f"\n--- {i+1} 回目の実行 ---")
    rating = 1 if i % 2 == 0 else 0
        
    if rating == 1:
        response_text = get_output_by_g4f(question)
    else:
        response_text = get_output_by_local_model(question)
        
    x.append(response_text)
    y.append(rating)
        
    print(f"question: {question}")
    print(f"rating:   {rating}")
    print(f"output:   {response_text}")




--- 1 回目の実行 ---
question: LLMを使った新しいアプリのアイデアをいくつか出してください
rating:   1
output:   もちろんです！以下は、LLM（大規模言語モデル）を活用した新しいアプリのアイデアです。

1. **パーソナライズド学習アプリ**:
   学習者の理解度や興味に基づいて、カスタマイズされた学習プランや教材を提供するアプリ。ユーザーが質問をすると、LLMがそれに対する説明や関連資料を生成し、学習をサポートします。

2. **ライティングアシスタント**:
   ユーザーが書いた文章を分析し、文法やスタイルの改善点を提案するアプリ。さらに、特定のトピックについてのアイデアや構成を提供する機能も搭載。

3. **メンタルヘルスサポートチャットボット**:
   ユーザーが気軽に相談できるチャットボット。感情や悩みを共有すると、それに対する共感的な応答やリソースを提供し、必要に応じて専門家への相談を勧める機能を持つ。

4. **旅行プランナー**:
   ユーザーの好みや予算に基づいて、旅行の行き先やアクティビティを提案するアプリ。ユーザーが希望する条件を入力すると、LLMが最適なプランを生成します。

5. **自動翻訳と文化ガイド**:
   日常会話やビジネス文書をリアルタイムで翻訳するアプリ。翻訳だけでなく、文化的な背景や言語のニュアンスについても説明を加えることで、より深い理解を促します。

6. **クリエイティブストーリー生成アプリ**:
   ユーザーが入力したキーワードやテーマに基づいて、短編小説や詩を生成するアプリ。ユーザーが選んだスタイルやジャンルに応じて、異なるアプローチを提案します。

7. **健康管理アシスタント**:
   ユーザーの健康データ（食事、運動、睡眠など）を分析し、健康的な生活習慣を提案するアプリ。食事のレシピやエクササイズプランを生成し、モチベーションを高めるサポートをします。

8. **パーソナルファイナンスアドバイザー**:
   ユーザーの収入や支出に基づいて、予算管理や投資のアドバイスを提供するアプリ。LLMがユーザーの状況を理解し、具体的なアクションプランを提案します。

これらのアイデアは、LLMの能力を活かしてユーザーのニーズに応えるためのものです

In [26]:
import json

# 保存ファイル名
output_file = "x_y_data.json"

# データを辞書形式で保存
data = {"x": x, "y": y}

# JSONファイルに書き込む
with open(output_file, "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print(f"データが {output_file} に保存されました！")

データが x_y_data.json に保存されました！
